In [1]:
!pip uninstall tensorflow -y
!pip install "tensorflow<=2.4.1"

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 394.3 MB 14 kB/s 
     |████████████████████████████████| 2.9 MB 37.2 MB/s 
     |████████████████████████████████| 3.8 MB 33.9 MB/s 
     |████████████████████████████████| 462 kB 63.0 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.41.0
    Uninstalling grpcio-1.41.0:
      Successfully uninstalled grpcio-1.41.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.chdir('/content/gdrive/MyDrive/Custom_Mask_RCNN/Mask_RCNN/samples/Fruit')

In [4]:
!pwd

/content/gdrive/MyDrive/Custom_Mask_RCNN/Mask_RCNN/samples/Fruit


In [5]:
import io
import base64
from IPython.display import HTML

video = io.open('/content/gdrive/MyDrive/Custom_Mask_RCNN/Mask_RCNN/samples/Fruit/custom_fruit.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="50%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [6]:
import sys

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

from mrcnn.config import Config
import mrcnn.model as modellib
import mrcnn.visualize
import cv2
import numpy as np

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to Fruit trained weights
Fruit_WEIGHTS_PATH = "/content/gdrive/MyDrive/Custom_Mask_RCNN/mask_rcnn_fruit_0070.h5"  # TODO: update this path

CLASS_NAMES = ['BG', 'Tomato', 'Oriental melon', 'Apple']  # TODO: update Class Name, Labelme 에서 정의한 Class 순서대로 나열

# 모델 구성 매개변수 준비
class FruitConfig(Config):
    # Give the configuration a recognizable name
    NAME = "Fruit"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = len(CLASS_NAMES)  # Background + Fruit

    # Skip detections with < 80% confidence
    DETECTION_MIN_CONFIDENCE = 0.8

# Mask R-CNN 모델 아키텍처 구축
model = modellib.MaskRCNN(mode="inference",    # "training" 또는 "inference" 
                          config=FruitConfig(),
                          model_dir=MODEL_DIR)

# pre-training 된 model weights load
model.load_weights(filepath=Fruit_WEIGHTS_PATH, 
                   by_name=True)

Instructions for updating:
Use fn_output_signature instead


In [7]:
def random_colors(N):
    #np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

colors = random_colors(len(CLASS_NAMES))

class_dict = {
    name : color for name, color in zip(CLASS_NAMES, colors)
}  

In [8]:
def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

In [9]:
def display_instances(image, boxes, masks, class_ids, class_names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    # Number of instances
    n_instances = boxes.shape[0]

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = class_names[class_ids[i]]
        color = class_dict[label]
        score =  scores[i] if scores is not None else None
        caption = "{} {:.3f}".format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
        image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.5, color, 2) 
 
    return image

In [10]:
input_video = '/content/gdrive/MyDrive/Custom_Mask_RCNN/Mask_RCNN/samples/Fruit/custom_fruit.mp4'
cap = cv2.VideoCapture(input_video)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

# Recording Video
fps = 20
width = int(cap.get(3))
height = int(cap.get(4))
fcc = cv2.VideoWriter_fourcc('D', 'I', 'V', 'X')
writer = cv2.VideoWriter("/content/output_custom_fruit.mp4", fcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        break

    # Object Detection
    r = model.detect([frame], verbose=0)
    r = r[0]

    # 예측결과 시각화
    frame = display_instances(image=frame, 
                              boxes=r['rois'], 
                              masks=r['masks'], 
                              class_ids=r['class_ids'], 
                              class_names=CLASS_NAMES, 
                              scores=r['scores'])
       
    # disk 에 frame 을 write 합니다.
    if writer is not None:
        writer.write(frame)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


NO INSTANCES TO DISPLAY
NO INSTANCES TO DISPLAY
--(!) No captured frame -- Break!
